<a href="https://colab.research.google.com/github/abhishekkm8088/Pyspark-Abhi/blob/main/Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello world")


hello world


##LINUX BASIC

In [11]:
!cat /etc/os-release
!uname -a


PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
Linux ff3c83f75672 6.1.123+ #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [13]:
!pwd


/content


In [14]:
!whoami

root


##Pyspark Basics and dataframes

In [17]:
pip install pyspark


In [19]:
from pyspark.sql import SparkSession

In [21]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [26]:
#create Dataframes
data =[("hello", "world")]
columns = ["word1", "word2"]

df = spark.createDataFrame(data, columns)

In [28]:
df.show()

+-----+-----+
|word1|word2|
+-----+-----+
|hello|world|
+-----+-----+



##Basic Transformation and action

In [30]:
columns = ["name", "department", "salary"]
data = [
    ("John", "Sales", 3000),
    ("Jane", "Finance", 4000),
    ("Mike", "Sales", 3500),
    ("Alice", "Finance", 3800),
    ("Bob", "IT", 4500)
]
df = spark.createDataFrame(data, columns)


In [32]:
df.show()

+-----+----------+------+
| name|department|salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [34]:
#filter: employees with salary > 3500
df_filtered = df.filter(df.salary > 3500)
df_filtered.show()

+-----+----------+------+
| name|department|salary|
+-----+----------+------+
| Jane|   Finance|  4000|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [36]:
#GroupBY and Aggregate: Average salary by department
df_agg = df.groupBy("department").agg({"salary": "avg"})
df_agg.show()

+----------+-----------+
|department|avg(salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [40]:
# Add new column : salary with bonos (10%)
from pyspark.sql.functions import col
exp=col("salary") * 1.1
df_with_bonus = df.withColumn("salary_with_bonus", exp)
df_with_bonus.show()

+-----+----------+------+------------------+
| name|department|salary| salary_with_bonus|
+-----+----------+------+------------------+
| John|     Sales|  3000|3300.0000000000005|
| Jane|   Finance|  4000|            4400.0|
| Mike|     Sales|  3500|3850.0000000000005|
|Alice|   Finance|  3800|            4180.0|
|  Bob|        IT|  4500|            4950.0|
+-----+----------+------+------------------+



In [42]:
from pyspark.sql.functions import col,upper,lower,concat_ws,when

df.show()

+-----+----------+------+
| name|department|salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [44]:
#Change case Transformation
df_upper = df.withColumn("name_upper", upper(col("name")))
df.lower=df.withColumn("name_lower", lower(col("name")))
df_upper.show()
df.lower.show()

+-----+----------+------+----------+
| name|department|salary|name_upper|
+-----+----------+------+----------+
| John|     Sales|  3000|      JOHN|
| Jane|   Finance|  4000|      JANE|
| Mike|     Sales|  3500|      MIKE|
|Alice|   Finance|  3800|     ALICE|
|  Bob|        IT|  4500|       BOB|
+-----+----------+------+----------+

+-----+----------+------+----------+
| name|department|salary|name_lower|
+-----+----------+------+----------+
| John|     Sales|  3000|      john|
| Jane|   Finance|  4000|      jane|
| Mike|     Sales|  3500|      mike|
|Alice|   Finance|  3800|     alice|
|  Bob|        IT|  4500|       bob|
+-----+----------+------+----------+



In [46]:
#concatenate columns
df_concat = df.withColumn("name_department", concat_ws(" - ", col("name"), col("department")))
df_concat.show()

+-----+----------+------+---------------+
| name|department|salary|name_department|
+-----+----------+------+---------------+
| John|     Sales|  3000|   John - Sales|
| Jane|   Finance|  4000| Jane - Finance|
| Mike|     Sales|  3500|   Mike - Sales|
|Alice|   Finance|  3800|Alice - Finance|
|  Bob|        IT|  4500|       Bob - IT|
+-----+----------+------+---------------+



In [48]:
#string length of names in new DF
from pyspark.sql.functions import length
df_length = df.withColumn("name_length", length(col("name")))
df_length.show()

+-----+----------+------+-----------+
| name|department|salary|name_length|
+-----+----------+------+-----------+
| John|     Sales|  3000|          4|
| Jane|   Finance|  4000|          4|
| Mike|     Sales|  3500|          4|
|Alice|   Finance|  3800|          5|
|  Bob|        IT|  4500|          3|
+-----+----------+------+-----------+



In [51]:
#add some conditional columns (salary, category) high medium and low
df_with_category = df.withColumn(
    "category",
    when(col("salary") >= 4000, "High")
    .when((col("salary") >= 3500) & (col("salary") < 4000), "Medium")
    .otherwise("Low")
)
df_with_category.show()




+-----+----------+------+--------+
| name|department|salary|category|
+-----+----------+------+--------+
| John|     Sales|  3000|     Low|
| Jane|   Finance|  4000|    High|
| Mike|     Sales|  3500|  Medium|
|Alice|   Finance|  3800|  Medium|
|  Bob|        IT|  4500|    High|
+-----+----------+------+--------+



In [52]:
# Rename Column (Salary --> Base_salary)
df_renamed = df.withColumnRenamed("salary", "base_salary")
df_renamed.show()

+-----+----------+-----------+
| name|department|base_salary|
+-----+----------+-----------+
| John|     Sales|       3000|
| Jane|   Finance|       4000|
| Mike|     Sales|       3500|
|Alice|   Finance|       3800|
|  Bob|        IT|       4500|
+-----+----------+-----------+

